In [2]:
#import joblib  
#model = joblib.load('model.pkl')

import bz2file as bz2
import pickle
model = pickle.load(bz2.BZ2File('../model/model.pbz2', 'rb'))

In [3]:
import pandas as pd

In [4]:
single_df = pd.DataFrame([['MOOCA', 'Apple', 'DOMINGO', 'NOITE', 45, 'HOMEM']], columns=['BAIRRO', 'MARCA_OBJETO', 'DIA_SEMANA', 'PERIODO', 'IDADE', 'SEXO'])

In [6]:
X_test = pd.read_csv("../data/base_to_single_predict.csv")
X_test = pd.concat([X_test, single_df])

In [7]:
X_test["BAIRRO"] = X_test["BAIRRO"].astype("category")
X_test["MARCA_OBJETO"] = X_test["MARCA_OBJETO"].astype("category")
X_test["DIA_SEMANA"] = X_test["DIA_SEMANA"].astype("category")
X_test["SEXO"] = X_test["SEXO"].astype("category")
X_test["PERIODO"] = X_test["PERIODO"].astype("category")

In [9]:
prev = model.predict_proba(X_test)

In [11]:
prev

array([[0.054043  , 0.05405313, 0.05434847, ..., 0.06165455, 0.06152413,
        0.48872638],
       [0.05594808, 0.05595857, 0.05626432, ..., 0.06000696, 0.06288915,
        0.4786366 ],
       [0.05527072, 0.05528107, 0.05558313, ..., 0.06498084, 0.05764281,
        0.47223347],
       ...,
       [0.05671613, 0.05672676, 0.05703671, ..., 0.06509287, 0.06430768,
        0.4547021 ],
       [0.05629808, 0.05630863, 0.0566163 , ..., 0.06643684, 0.06925853,
        0.45143783],
       [0.05397427, 0.05398438, 0.05427935, ..., 0.06166743, 0.05994694,
        0.49157128]], dtype=float32)

In [13]:
response = prev[-1]

In [43]:
single_pred = pd.DataFrame(response)

In [44]:
single_pred = pd.DataFrame([[
    single_pred[0][0],
    single_pred[0][1],
    single_pred[0][2],
    single_pred[0][3],
    single_pred[0][4],
    single_pred[0][5],
    single_pred[0][6],
    single_pred[0][7],
    single_pred[0][8],
    single_pred[0][9]
]], columns=[
    "Escolas / Universidade", 
    "Escritórios", 
    "Saúde",
    "Entreterimento",
    "Alimentação",
    "Transporte Privado",
    "Estabelecimentos Comerciais / Industrias",
    "Residência / Hospedagem",
    "Transporte Público",
    "Via Pública"
])

In [45]:
single_pred

,Escolas / Universidade,Escritórios,Saúde,Entreterimento,Alimentação,Transporte Privado,Estabelecimentos Comerciais / Industrias,Residência / Hospedagem,Transporte Público,Via Pública
0,0.053974,0.053984,0.054279,0.054472,0.054505,0.059353,0.056247,0.061667,0.059947,0.491571


In [46]:
single_pred *= 100

In [47]:
single_pred

,Escolas / Universidade,Escritórios,Saúde,Entreterimento,Alimentação,Transporte Privado,Estabelecimentos Comerciais / Industrias,Residência / Hospedagem,Transporte Público,Via Pública
0,5.397427,5.398438,5.427935,5.447151,5.450509,5.935262,5.624709,6.166743,5.994694,49.157127


In [48]:
single_pred = single_pred.sort_values(single_pred.last_valid_index(), axis=1)

In [49]:
single_pred

,Escolas / Universidade,Escritórios,Saúde,Entreterimento,Alimentação,Estabelecimentos Comerciais / Industrias,Transporte Privado,Transporte Público,Residência / Hospedagem,Via Pública
0,5.397427,5.398438,5.427935,5.447151,5.450509,5.624709,5.935262,5.994694,6.166743,49.157127


In [50]:
result = single_pred.to_json(orient="split")
from json import loads, dumps
parsed = loads(result)

In [51]:
parsed

{'columns': ['Escolas / Universidade',
  'Escritórios',
  'Saúde',
  'Entreterimento',
  'Alimentação',
  'Estabelecimentos Comerciais / Industrias',
  'Transporte Privado',
  'Transporte Público',
  'Residência / Hospedagem',
  'Via Pública'],
 'index': [0],
 'data': [[5.3974270821,
   5.3984384537,
   5.4279351234,
   5.4471507072,
   5.4505085945,
   5.6247091293,
   5.9352622032,
   5.9946937561,
   6.1667432785,
   49.1571273804]]}

In [52]:
response = []
for x in range(10):
  data = {
            "position":x,
            "label":parsed['columns'][x],
            "value":parsed['data'][0][x]
        }
  
  response.append(data)

In [53]:
response

[{'position': 0, 'label': 'Escolas / Universidade', 'value': 5.3974270821},
 {'position': 1, 'label': 'Escritórios', 'value': 5.3984384537},
 {'position': 2, 'label': 'Saúde', 'value': 5.4279351234},
 {'position': 3, 'label': 'Entreterimento', 'value': 5.4471507072},
 {'position': 4, 'label': 'Alimentação', 'value': 5.4505085945},
 {'position': 5,
  'label': 'Estabelecimentos Comerciais / Industrias',
  'value': 5.6247091293},
 {'position': 6, 'label': 'Transporte Privado', 'value': 5.9352622032},
 {'position': 7, 'label': 'Transporte Público', 'value': 5.9946937561},
 {'position': 8, 'label': 'Residência / Hospedagem', 'value': 6.1667432785},
 {'position': 9, 'label': 'Via Pública', 'value': 49.1571273804}]

In [54]:
import matplotlib.pyplot as plt
import xgboost as xgb

In [ ]:
#pip install graphviz

In [ ]:
fig, ax = plt.subplots(figsize=(30,30))
xgb.plot_tree(model, num_trees=4, ax=ax)
plt.show()